# Result comparison for AL

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
def load_mses(folder):
    """
    Load the mses of a certain model, being called by the aggregate function below
    """
    test_mse = np.load(os.path.join(folder, 'test_mse.npy'))
    train_mse = np.load(os.path.join(folder, 'train_mse.npy'))
    selected_after_train_mse = np.load(os.path.join(folder, 'mse_selected_after_train.npy'))
    return test_mse, train_mse, selected_after_train_mse

def load_ensemble_mses(folder, num_trails, avg_test_mse_mat=None, avg_train_mse_mat=None, avg_selected_after_train_mse_mat=None):
    """
    Load multiple mse file from different trails from num_trails parameter and 
    aggregate them by taking mean
    """
    if avg_test_mse_mat is None:
        avg_test_mse_mat, avg_train_mse_mat, avg_selected_after_train_mse_mat = load_matrix(folder, num_trails)
    # Take the average
    return np.mean(avg_test_mse_mat, axis=1), np.mean(avg_train_mse_mat, axis=1), np.mean(avg_selected_after_train_mse_mat, axis=1)

def load_matrix(folder, num_trails):
    """
    As the error is prone to outlier in the x value, therefore we can not simply do average (the outliers have explicitly higher weights)
    Here we are showing the ratio of training and testing
    """
    avg_test_mse_mat, avg_train_mse_mat, avg_selected_after_train_mse_mat = [], [], []
    for i in range(num_trails):
        # Replace to get the new folder name
        folder_name = folder.replace('trail_0','trail_{}'.format(i))
        # From folder name get all the mse lists
        test_mse, train_mse, selected_after_train_mse = load_mses(folder_name)
        # Average them
        avg_test_mse_mat.append(test_mse)
        avg_train_mse_mat.append(train_mse)
        avg_selected_after_train_mse_mat.append(selected_after_train_mse)
    avg_test_mse_mat = np.array(avg_test_mse_mat).transpose()
    avg_train_mse_mat = np.array(avg_train_mse_mat).transpose()
    avg_selected_after_train_mse_mat = np.array(avg_selected_after_train_mse_mat).transpose()
    print('shape = ', np.shape(avg_test_mse_mat))
    return avg_test_mse_mat, avg_train_mse_mat, avg_selected_after_train_mse_mat

def load_coeff(folder, num_trails, mode):
    """
    Loading the variance coefficient with real MSE
    """
    var_coeff_mat = []
    if mode == 'R':
        npy_file = 'var_mse_coreff.npy'
    elif mode == 'Tau':
        npy_file = 'var_mse_tau.npy'
    for i in range(num_trails):
        folder_name = folder.replace('trail_0','trail_{}'.format(i))
        file_name = os.path.join(folder_name, npy_file)
        coef_list = np.load(file_name)
        var_coeff_mat.append(np.copy(coef_list))
    return var_coeff_mat

def load_kendal_tau(folder, num_trails):
    """
    Loading the variance Kendal's Tau 
    """
    return load_coeff(folder, num_trails, mode='Tau')

def load_var_coeff(folder, num_trails):
    """
    Loading the variance coefficient with real MSE
    """
    return load_coeff(folder, num_trails, mode='R')


In [17]:
num_trails = 20
if num_trails <= 10:
    alpha = 0.5
else:
    alpha = 0.2
plot_var = True
#MSE_retrain_False_bs_1024_pool_10_dx_5_x0_20_nmod_5
#folder = 'results/fig'
folder = '/home/sr365/NAAL/results/fig/retrain_True'
#folder = '/home/sr365/NAAL/results/fig/retrain_False'
#folder = 'results/fig/retrain_true_al_step_50/x0_20'

#folder_list = ['/home/sr365/NAAL/results/fig/retrain_True', '/home/sr365/NAAL/results/fig/retrain_False','results/fig/retrain_true_al_step_50/x0_20']
#folder_list = ['/home/sr365/NAAL/results/single_model']
folder_list = ['results/30_sine_nmod']
for folder in folder_list:
    for file in os.listdir(folder):
        cur_folder = os.path.join(folder, file)
        # Makue sure this is a folder
        if not os.path.isdir(cur_folder):
            continue
        # Only start from the M SE folders and look for comparison
        if not 'MSE' in cur_folder or not 'trail_0' in cur_folder:# or 'nmod_20' in cur_folder:
            continue
        
        VAR_test_mse_mat = None # This is for the case where it is not called

        # The reading part
        #MSE_test_mse, MSE_train_mse, MSE_selected_after_train_mse = load_mses(cur_folder)
        MSE_test_mse_mat, MSE_train_mse_mat, MSE_selected_after_train_mse_mat = load_matrix(cur_folder, num_trails)
        MSE_test_mse, MSE_train_mse, MSE_selected_after_train_mse = load_ensemble_mses(cur_folder, num_trails, MSE_test_mse_mat, MSE_train_mse_mat, MSE_selected_after_train_mse_mat)
        
        random_folder = cur_folder.replace('MSE','Random')
        #RD_test_mse, RD_train_mse, RD_selected_after_train_mse = load_mses(random_folder)
        Random_test_mse_mat, Random_train_mse_mat, Random_selected_after_train_mse_mat = load_matrix(random_folder, num_trails)
        RD_test_mse, RD_train_mse, RD_selected_after_train_mse = load_ensemble_mses(random_folder, num_trails, Random_test_mse_mat, Random_train_mse_mat, Random_selected_after_train_mse_mat)

        if plot_var:
            # Var plotting
            var_folder = cur_folder.replace('MSE','VAR')
            VAR_test_mse_mat, VAR_train_mse_mat, VAR_selected_after_train_mse_mat = load_matrix(var_folder, num_trails)
            VAR_test_mse, VAR_train_mse, VAR_selected_after_train_mse = load_ensemble_mses(var_folder, num_trails, VAR_test_mse_mat, VAR_train_mse_mat, VAR_selected_after_train_mse_mat)

        dx = int(file.split('_dx_')[-1].split('_')[0])
        x0 = int(file.split('_x0_')[-1].split('_')[0])
        print('for {}, dx = {}, x0 = {}'.format(file, dx, x0))
        # Use the xlabel with number of data points
        x_label = np.array(range(len(MSE_test_mse))) * dx + x0
        save_name = os.path.join(folder, file.replace('MSE_','').replace('_trail_0_',''))
        ##########################################
        # Plotting various MSE metrics together #
        ##########################################
        f = plt.figure(figsize=[8, 4])
        #ax1 = plt.subplot(211)
        plt.plot(x_label, MSE_test_mse, '-x', c='tab:blue', label='MSE test')
        plt.plot(x_label, RD_test_mse, '-x', c='tab:orange', label='RD test')
        plt.plot(x_label, MSE_train_mse, '--x', c='tab:blue', linewidth=2, label='MSE train')
        plt.plot(x_label, RD_train_mse, '--x', c='tab:orange', linewidth=2, label='RD train')
        if plot_var:
            plt.plot(x_label, VAR_test_mse, '-x', c='tab:green', label='VAR test')
            plt.plot(x_label, VAR_train_mse, '--x', c='tab:green', linewidth=2, label='VAR train')
        plt.legend()
        plt.yscale('log')
        plt.xlabel('num_train')
        plt.ylabel('MSE')
        plt.title(os.path.basename(save_name))
        plt.savefig(save_name+'.png')
        plt.clf()

        ###################################
        # Plotting AL/Raondom Ratio plot #
        ###################################
        def plot_ratio(num, denom, x_value, ylabel, save_name):
            # Get the MSE/Random ratio
            test_mse_ratio = num / denom
            test_mse_ratio = np.log(test_mse_ratio)
            percent_better = np.mean(test_mse_ratio < 0)
            #test_mse_ratio[test_mse_ratio < 1] = -np.log(1/test_mse_ratio[test_mse_ratio < 1])
            #test_mse_ratio[test_mse_ratio >= 1] = np.log(test_mse_ratio[test_mse_ratio >= 1])
            # Plot the ratio figure
            f = plt.figure(figsize=[8, 4])
            for i in range(num_trails):
                plt.plot(x_value, test_mse_ratio[:, i],'x')
            plt.plot(x_value, np.zeros_like(x_label),'--r', label='ref: {:.1f}%'.format(percent_better*100))
            plt.legend()
            #plt.ylim([-2, 2])
            #plt.yscale('log')
            plt.xlabel('num_train')
            plt.ylabel(ylabel)
            plt.title(os.path.basename(save_name))
            plt.savefig(save_name + 'ratio_plot.png')
            plt.clf()
        
        #######################################
        # Plotting Test MSE comparison plot #
        #######################################
        def plot_test_MSE_plot(x_label, MSE_test_mse_mat, Random_test_mse_mat, save_name, VAR_test_mse_mat=None, plot_var=False):
            # Get the comparison plot for all
            f = plt.figure(figsize=[8, 4])
            # For easy legend
            plt.plot(x_label, MSE_test_mse_mat[:, 0], '-', alpha=alpha,  c='tab:blue', label='AL')
            plt.plot(x_label, Random_test_mse_mat[:, 0], '-', alpha=alpha,  c='tab:orange', label='Random')
            if plot_var:
                plt.plot(x_label, VAR_test_mse_mat[:, 0], '-', alpha=alpha,  c='tab:green', label='VAR')
            for i in range(1, num_trails):
                plt.plot(x_label, MSE_test_mse_mat[:, i], '-', alpha=alpha,  c='tab:blue')
                plt.plot(x_label, Random_test_mse_mat[:, i], '-', alpha=alpha,  c='tab:orange')
                if plot_var:
                    plt.plot(x_label, VAR_test_mse_mat[:, i], '-', alpha=alpha,  c='tab:green')
            plt.xlabel('num_train')
            plt.ylabel('MSE-(VAR) / Random Ratio')
            plt.yscale('log')
            plt.legend()
            plt.title(os.path.basename(save_name))
            plt.savefig(save_name + 'test_loss_overlay.png')

        ###################################
        # # Plotting VAR correlation plot #
        ###################################
        def plot_coref(cur_folder, num_trails, save_name, coef_loader, coef_name):
            f = plt.figure(figsize=[8, 4])
            var_folder = cur_folder.replace('MSE','VAR')
            coeff = coef_loader(var_folder, num_trails)
            for i in range(num_trails):
                plt.plot(coeff[i], 'x', alpha=0.2)
            plt.ylim([0, 1])
            plt.xlabel('epoch')
            plt.ylabel('correlation')
            plt.savefig(save_name + coef_name + '_cor.png')

        #######################################################
        # Plotting the Kendall's tau for correlation analysis #
        #######################################################
        # One-liner calling functions
        plot_ratio(MSE_test_mse_mat, Random_test_mse_mat, x_label, ylabel='MSE / Random Ratio', save_name=save_name + 'MSE_RD_')                # Plot the ratio plot between MSE and random
        plot_test_MSE_plot(x_label, MSE_test_mse_mat, Random_test_mse_mat, save_name, VAR_test_mse_mat=VAR_test_mse_mat, plot_var=plot_var)     # Plot the test MSE line plot

        if plot_var:    # For variance experiments 
            plot_ratio(VAR_test_mse_mat, Random_test_mse_mat, x_label, ylabel='VAR / Random Ratio', save_name=save_name + 'VAR_RD_')            # Plot the ratio plot between VAR and random
            plot_coref(cur_folder, num_trails, save_name, coef_loader=load_var_coeff, coef_name='VAR')                                                                                       # Plot the VAR R coeff plot
            plot_coref(cur_folder, num_trails, save_name, coef_laoder=load_kendal_tau, coef_name='Tau')                                         # Plot the VAR Tau coeff plot   


        

        
        plt.clf()
        

shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_True_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_10_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_False_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_5_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_True_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_5_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_False_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_20_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_False_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_10_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_True_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_50_trail_0, dx = 5, x0 = 10
20
shape =  (38, 20)
shape =  (38, 20)
shape =  (38, 20)
for MSE_retrain_True_bs_1024_pool_100_dx_5_step_38_x0_10_nmod_2

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

...

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

# Testing the numpy random seed

In [8]:
import numpy as np
np.random.seed(1)
print(np.random.uniform(10))
print(np.random.uniform(0, 5, 10))


6.246801957676833
[3.60162247e+00 5.71874087e-04 1.51166286e+00 7.33779454e-01
 4.61692974e-01 9.31301057e-01 1.72780364e+00 1.98383737e+00
 2.69408367e+00 2.09597257e+00]


In [7]:
np.random.seed(1)
print(np.random.uniform(10))
print(np.random.uniform(0, 5, 10))

6.246801957676833
[3.60162247e+00 5.71874087e-04 1.51166286e+00 7.33779454e-01
 4.61692974e-01 9.31301057e-01 1.72780364e+00 1.98383737e+00
 2.69408367e+00 2.09597257e+00]
